In [1]:
import pandas as pd
reviews = pd.read_csv("googleplaystore_user_reviews.csv")

In [2]:
print(reviews.head())

                     App                                  Translated_Review  \
0  10 Best Foods for You  I like eat delicious food. That's I'm cooking ...   
1  10 Best Foods for You    This help eating healthy exercise regular basis   
2  10 Best Foods for You                                                NaN   
3  10 Best Foods for You         Works great especially going grocery store   
4  10 Best Foods for You                                       Best idea us   

  Sentiment  Sentiment_Polarity  Sentiment_Subjectivity  
0  Positive                1.00                0.533333  
1  Positive                0.25                0.288462  
2       NaN                 NaN                     NaN  
3  Positive                0.40                0.875000  
4  Positive                1.00                0.300000  


In [3]:
print("\nREVIEWS file:", reviews.shape)
print(reviews.columns.tolist())


REVIEWS file: (64295, 5)
['App', 'Translated_Review', 'Sentiment', 'Sentiment_Polarity', 'Sentiment_Subjectivity']


In [4]:
print("\nTop missing counts (reviews):")
print(reviews.isna().sum().sort_values(ascending=False).head(10))


Top missing counts (reviews):
Translated_Review         26868
Sentiment                 26863
Sentiment_Polarity        26863
Sentiment_Subjectivity    26863
App                           0
dtype: int64


In [5]:
# Drop rows where review text is missing
reviews_clean = reviews.dropna(subset=['Translated_Review']).copy()

In [6]:
# Keep only useful columns
reviews_clean = reviews_clean[['App','Translated_Review','Sentiment','Sentiment_Polarity','Sentiment_Subjectivity']]

In [7]:
# Reset index
reviews_clean.reset_index(drop=True, inplace=True)

In [8]:
print("After cleaning:", reviews_clean.shape)

After cleaning: (37427, 5)


In [9]:
print(reviews_clean.head(10))

                     App                                  Translated_Review  \
0  10 Best Foods for You  I like eat delicious food. That's I'm cooking ...   
1  10 Best Foods for You    This help eating healthy exercise regular basis   
2  10 Best Foods for You         Works great especially going grocery store   
3  10 Best Foods for You                                       Best idea us   
4  10 Best Foods for You                                           Best way   
5  10 Best Foods for You                                            Amazing   
6  10 Best Foods for You                               Looking forward app,   
7  10 Best Foods for You              It helpful site ! It help foods get !   
8  10 Best Foods for You                                          good you.   
9  10 Best Foods for You  Useful information The amount spelling errors ...   

  Sentiment  Sentiment_Polarity  Sentiment_Subjectivity  
0  Positive                1.00                0.533333  
1  Positive   

In [10]:
# Count sentiment types per app
sentiment_summary = reviews_clean.groupby(['App','Sentiment']).size().unstack(fill_value=0)

In [11]:
# Calculate total reviews and positivity %
sentiment_summary['Total_Reviews'] = sentiment_summary.sum(axis=1)
sentiment_summary['Positive%'] = (sentiment_summary['Positive'] / sentiment_summary['Total_Reviews']) * 100


In [12]:
# Reset index for clean table
sentiment_summary = sentiment_summary.reset_index()

In [13]:
print(sentiment_summary.head(10))

Sentiment                                               App  Negative  \
0                                     10 Best Foods for You        10   
1                          104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室         1   
2                                                      11st         7   
3                                1800 Contacts - Lens Store         6   
4                           1LINE – One Line with One Touch         8   
5          2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif         1   
6                              21-Day Meditation Experience        10   
7                       2Date Dating App, Love and matching         7   
8                               2GIS: directory & navigator         6   
9                                                 2RedBeans         2   

Sentiment  Neutral  Positive  Total_Reviews  Positive%  
0               22       162            194  83.505155  
1                8        31             40  77.500000  
2                9       

In [14]:
# Export sentiment summary to CSV
sentiment_summary.to_csv("sentiment_summary.csv", index=False)

In [15]:
apps = pd.read_csv("googleplaystore.csv")

In [16]:
print(apps.shape)
print(apps.columns)
print(apps.head())

(10841, 13)
Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')
                                                 App        Category  Rating  \
0     Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN     4.1   
1                                Coloring book moana  ART_AND_DESIGN     3.9   
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN     4.7   
3                              Sketch - Draw & Paint  ART_AND_DESIGN     4.5   
4              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN     4.3   

  Reviews  Size     Installs  Type Price Content Rating  \
0     159   19M      10,000+  Free     0       Everyone   
1     967   14M     500,000+  Free     0       Everyone   
2   87510  8.7M   5,000,000+  Free     0       Everyone   
3  215644   25M  50,000,000+  Free     0           Teen   
4     967  2.8M  

In [17]:
# Drop duplicates
apps_clean = apps.drop_duplicates().copy()

In [18]:
# Remove rows with missing App name
apps_clean = apps_clean.dropna(subset=['App'])


In [19]:
# Reset index
apps_clean.reset_index(drop=True, inplace=True)

In [20]:
print("After cleaning:", apps_clean.shape)

After cleaning: (10358, 13)


In [22]:
print(apps_clean['Installs'].unique()[:20])  # show first 20 unique values

['10,000+' '500,000+' '5,000,000+' '50,000,000+' '100,000+' '50,000+'
 '1,000,000+' '10,000,000+' '5,000+' '100,000,000+' '1,000,000,000+'
 '1,000+' '500,000,000+' '50+' '100+' '500+' '10+' '1+' '5+' '0+']


In [23]:
# Remove rows with bad values (like 'Free') in 'Installs'
apps_clean = apps_clean[apps_clean['Installs'].str.contains(r'^\d+[+,]*$', na=False)]

In [24]:
# Remove commas and plus signs
apps_clean['Installs'] = apps_clean['Installs'].str.replace('[+,]', '', regex=True)

In [25]:
# Convert to int
apps_clean['Installs'] = apps_clean['Installs'].astype(int)

In [26]:
apps_clean['Price'] = apps_clean['Price'].str.replace('$', '', regex=False) 

In [27]:
# Clean Size (convert M to float, K to float, handle 'Varies with device')
def clean_size(size):
    if 'M' in size:
        return float(size.replace('M',''))
    elif 'k' in size:
        return float(size.replace('k','')) / 1000
    elif size == 'Varies with device':
        return None
    else:
        return None

apps_clean['Size'] = apps_clean['Size'].apply(clean_size)

In [28]:
# Convert Rating to numeric
apps_clean['Rating'] = pd.to_numeric(apps_clean['Rating'], errors='coerce')

In [29]:
print(apps_clean[['App','Installs','Price','Size','Rating']].head(10))

                                                   App  Installs Price  \
431                                  Truth or Dare Pro        50  1.49   
432    Private Dating, Hide App- Blue for PrivacyHider       100  2.99   
433                               Ad Blocker for SayHi       100  3.99   
434  AMBW Dating App: Asian Men Black Women Interra...       100  7.99   
539             Cardi B Live Stream Video Chat - Prank       500     0   
542                     Chat Kids - Chat Room For Kids       100     0   
545                                        MouseMingle       100     0   
549                                       Awake Dating       100     0   
550                  Live Chat - Free Video Chat Rooms       500     0   
551           Meet With Strangers: Video Chat & Dating       500     0   

       Size  Rating  
431  20.000     NaN  
432   0.018     NaN  
433   1.200     NaN  
434  17.000     3.5  
539   3.400     4.4  
542   4.900     4.7  
545   3.900     2.7  
549  70.0

In [30]:
# Fill missing Rating with median
apps_clean['Rating'] = apps_clean['Rating'].fillna(apps_clean['Rating'].median())

In [31]:
# Fill missing Size with median (ignoring None)
apps_clean['Size'] = apps_clean['Size'].fillna(apps_clean['Size'].median())

In [32]:
# Fill missing Reviews with 0 (just in case)
apps_clean['Reviews'] = pd.to_numeric(apps_clean['Reviews'], errors='coerce')
apps_clean['Reviews'] = apps_clean['Reviews'].fillna(0)

In [33]:
print("Missing values after filling:")
print(apps_clean.isna().sum())

Missing values after filling:
App               0
Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              1
Price             0
Content Rating    0
Genres            0
Last Updated      0
Current Ver       2
Android Ver       0
dtype: int64


In [57]:
apps_clean.to_csv("apps_cleaned.csv", index=False)

print("Saved apps_cleaned.csv with shape:", apps_clean.shape)

Saved apps_cleaned.csv with shape: (1791, 13)


In [35]:
print("Merged dataset shape:", merged_data.shape)
print(merged_data.head(10))

Merged dataset shape: (0, 18)
Empty DataFrame
Columns: [App, Negative, Neutral, Positive, Total_Reviews, Positive%, Category, Rating, Reviews, Size, Installs, Type, Price, Content Rating, Genres, Last Updated, Current Ver, Android Ver]
Index: []


In [36]:
# Standardize App names in both datasets
sentiment_summary['App'] = sentiment_summary['App'].str.strip().str.lower()
apps_clean['App'] = apps_clean['App'].str.strip().str.lower()

In [37]:
# Merge again
merged_data = pd.merge(sentiment_summary, apps_clean, on="App", how="inner")

In [38]:
print("Merged dataset shape:", merged_data.shape)
print(merged_data.head(10))

Merged dataset shape: (0, 18)
Empty DataFrame
Columns: [App, Negative, Neutral, Positive, Total_Reviews, Positive%, Category, Rating, Reviews, Size, Installs, Type, Price, Content Rating, Genres, Last Updated, Current Ver, Android Ver]
Index: []


In [39]:
print("Unique Apps in sentiment_summary:", sentiment_summary['App'].nunique())

Unique Apps in sentiment_summary: 865


In [40]:
print("Unique Apps in apps_clean:", apps_clean['App'].nunique())

Unique Apps in apps_clean: 1788


In [41]:
# Find overlapping apps
common_apps = set(sentiment_summary['App']).intersection(set(apps_clean['App']))

In [42]:
print("Number of common apps:", len(common_apps))
print("Sample common apps:", list(common_apps)[:10])  # first 20

Number of common apps: 0
Sample common apps: []


In [43]:
print("Sample from sentiment_summary:")
print(sentiment_summary['App'].unique()[:20])


Sample from sentiment_summary:
['10 best foods for you' '104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室' '11st'
 '1800 contacts - lens store' '1line – one line with one touch'
 '2018emoji keyboard 😂 emoticons lite -sticker&gif'
 '21-day meditation experience' '2date dating app, love and matching'
 '2gis: directory & navigator' '2redbeans' '2ndline - second phone number'
 '30 day fitness challenge - workout at home' '365scores - live scores'
 '3d live neon weed launcher' '4 in a row'
 '4k wallpapers and ultra hd backgrounds'
 '591房屋交易-租屋、中古屋、新建案、實價登錄、別墅透天、公寓套房、捷運、買房賣房行情、房價房貸查詢' '591房屋交易-香港'
 '7 cups: anxiety & stress chat' '7 day food journal challenge']


In [44]:
print("\nSample from apps_clean:")
print(apps_clean['App'].unique()[:20])


Sample from apps_clean:
['truth or dare pro' 'private dating, hide app- blue for privacyhider'
 'ad blocker for sayhi'
 'ambw dating app: asian men black women interracial'
 'cardi b live stream video chat - prank' 'chat kids - chat room for kids'
 'mousemingle' 'awake dating' 'live chat - free video chat rooms'
 'meet with strangers: video chat & dating'
 'random chat app with strangers'
 'ost. zombies cast - new music and lyrics' 'dating white girls'
 'geeks dating' 'live chat - free video chat' 'house party - live chat'
 'fishing brain & boating maps marine' 'cam5678 video chat'
 'video chat live advices' 'pet lovers dating']


In [45]:
!pip install rapidfuzz

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 2.4 MB/s eta 0:00:01
   ------------------------------------- -- 1.6/1.7 MB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 2.8 MB/s  0:00:00


In [46]:
from rapidfuzz import process, fuzz

# Create mapping for top matches
mapping = {}
for app in sentiment_summary['App'].unique()[:50]:  # limit first 50 for testing
    match = process.extractOne(app, apps_clean['App'].unique(), scorer=fuzz.token_sort_ratio)
    if match and match[1] > 70:  # threshold for similarity
        mapping[app] = match[0]

print("Sample fuzzy matches:")
for k, v in list(mapping.items())[:20]:
    print(f"{k}  --->  {v}")

Sample fuzzy matches:
a+ mobile  --->  cb mobile
abc preschool free  --->  bg future school


In [47]:
from rapidfuzz import process, fuzz

# Build fuzzy mapping for all apps
mapping = {}
for app in sentiment_summary['App'].unique():
    match = process.extractOne(app, apps_clean['App'].unique(), scorer=fuzz.token_sort_ratio)
    if match and match[1] > 80:  # you can adjust threshold (70–85)
        mapping[app] = match[0]

# Show sample matches
print("Sample fuzzy matches:")
for k, v in list(mapping.items())[:20]:
    print(f"{k}  --->  {v}")

# Apply mapping
sentiment_summary['App_Matched'] = sentiment_summary['App'].map(mapping)

# Drop rows with no match
sentiment_summary = sentiment_summary.dropna(subset=['App_Matched'])

# Merge with apps_clean
merged_data = pd.merge(sentiment_summary, apps_clean, left_on="App_Matched", right_on="App", how="inner")

print("Merged dataset shape:", merged_data.shape)
print(merged_data.head(10))


Sample fuzzy matches:
all-in-one mahjong 3 free  --->  all-in-one mahjong 3
apk installer  --->  cp installer app
barcode scanner  --->  bi barcode scanner
best wallpapers qhd  --->  eh bee wallpapers hd
cbs news  --->  cb news
du recorder – screen recorder, video editor, live  --->  ay recorder - screen recorder, live, video editor
file explorer  --->  bl file explorer
file manager  --->  r file manager
go notifier  --->  cl notifier
hd camera  --->  dz camera
Merged dataset shape: (10, 20)
                                               App_x  Negative  Neutral  \
0                          all-in-one mahjong 3 free         0        0   
1                                      apk installer         6        3   
2                                    barcode scanner         0        6   
3                                best wallpapers qhd         4        1   
4                                           cbs news         0        1   
5  du recorder – screen recorder, video editor, live 

In [48]:
print("Merged dataset shape:", merged_data.shape)
print(merged_data.head(10))

Merged dataset shape: (10, 20)
                                               App_x  Negative  Neutral  \
0                          all-in-one mahjong 3 free         0        0   
1                                      apk installer         6        3   
2                                    barcode scanner         0        6   
3                                best wallpapers qhd         4        1   
4                                           cbs news         0        1   
5  du recorder – screen recorder, video editor, live        11        5   
6                                      file explorer         3        6   
7                                       file manager         3        5   
8                                        go notifier         8        5   
9                                          hd camera         0        1   

   Positive  Total_Reviews   Positive%  \
0         1              1  100.000000   
1        18             27   66.666667   
2        31      

In [49]:
merged_data = merged_data.drop(columns=['App_y'])
merged_data = merged_data.rename(columns={'App_x': 'App'})


In [51]:
print(merged_data.shape)
print(merged_data.columns)
merged_data.head(5)

(10, 19)
Index(['App', 'Negative', 'Neutral', 'Positive', 'Total_Reviews', 'Positive%',
       'App_Matched', 'Category', 'Rating', 'Reviews', 'Size', 'Installs',
       'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated',
       'Current Ver', 'Android Ver'],
      dtype='object')


,App,Negative,Neutral,Positive,Total_Reviews,Positive%,App_Matched,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,all-in-one mahjong 3 free,0,0,1,1,100.000000,all-in-one mahjong 3,FAMILY,4.4,38,16.000,100,Paid,0.99,Everyone,Board;Brain Games,"June 14, 2018",20180609,4.0 and up
1,apk installer,6,3,18,27,66.666667,cp installer app,BUSINESS,5.0,4,24.000,100,Free,0,Everyone,Business,"July 24, 2018",5.1.1,4.1 and up
2,barcode scanner,0,6,31,37,83.783784,bi barcode scanner,PRODUCTIVITY,4.6,0,0.473,10,Paid,0.99,Everyone,Productivity,"June 13, 2014",2.2,2.2 and up
3,best wallpapers qhd,4,1,30,35,85.714286,eh bee wallpapers hd,PERSONALIZATION,5.0,4,3.900,100,Free,0,Everyone,Personalization,"April 24, 2018",1.0,4.1 and up
4,cbs news,0,1,0,1,0.000000,cb news,FAMILY,5.0,7,20.000,50,Free,0,Everyone,Education,"September 10, 2016",1.2.0,4.1 and up


In [52]:
print(merged_data.describe())
print(merged_data['Category'].value_counts().head(10))

        Negative    Neutral   Positive  Total_Reviews   Positive%     Rating  \
count  10.000000  10.000000  10.000000      10.000000   10.000000  10.000000   
mean    3.500000   3.300000  16.000000      22.800000   55.848553   4.730000   
std     3.836955   2.359378  15.040686      19.141578   35.163174   0.266875   
min     0.000000   0.000000   0.000000       1.000000    0.000000   4.300000   
25%     0.000000   1.000000   1.500000       3.500000   35.671937   4.600000   
50%     3.000000   4.000000  16.000000      25.000000   64.215686   4.700000   
75%     5.500000   5.000000  27.750000      34.750000   80.941286   5.000000   
max    11.000000   6.000000  42.000000      58.000000  100.000000   5.000000   

         Reviews       Size    Installs  
count  10.000000  10.000000   10.000000  
mean   12.000000  11.149400  107.000000  
std    14.204851  11.368034  142.676791  
min     0.000000   0.421000   10.000000  
25%     1.750000   3.350000   50.000000  
50%     5.500000   5.300000

In [54]:
merged_data.to_csv("googleplay_sentiment_cleaned.csv", index=False)

In [55]:
match = process.extractOne(app, apps_clean['App'].unique(), scorer=fuzz.token_sort_ratio)
if match and match[1] > 50:  # lower from 70 → 50
    mapping[app] = match[0]


In [56]:
mapping = {}
for app in sentiment_summary['App'].unique():
    match = process.extractOne(app, apps_clean['App'].unique(), scorer=fuzz.token_sort_ratio)
    if match and match[1] > 50:  
        mapping[app] = match[0]

print(f"Total Matches: {len(mapping)}")


Total Matches: 10
